In [2]:
import os
os.environ["PYICEBERG_CONFIG"] = "./.pyiceberg.yaml"

In [4]:
import os
print(os.environ.get("PYICEBERG_CONFIG"))

./.pyiceberg.yaml


In [1]:
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema, NestedField
from pyiceberg.types import (
    StringType, IntegerType, LongType, StructType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField

# Load catalog Hive
# catalog = load_catalog("my_hive")

from pyiceberg.catalog.hive import HiveCatalog

from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "default",
    **{
        "type": "hive",
        "uri": "thrift://hive-metastore:9083",

        # Warehouse đặt trên S3 / MinIO
        "warehouse": "s3://warehouse/",

        # ===== CẤU HÌNH S3 ĐÚNG CÚ PHÁP =====
        "s3.endpoint": "http://minio:9000",
        "s3.access-key-id": "minioadmin",
        "s3.secret-access-key": "minioadmin",
        "s3.path-style-access": "true",
        "s3.region": "us-east-1",
    },
)

# Tên database và table
database_name = "test"
table_name = "test_pyiceberg"

# Tạo schema
schema = Schema(
    NestedField(1, "id", LongType(), required=True),
    NestedField(2, "name", StringType(), required=False),
    NestedField(3, "age", IntegerType(), required=False),
    NestedField(4, "department", StringType(), required=False),
)

# Partition theo department
spec = PartitionSpec(
    PartitionField(
        source_id=4,              # ID của cột "department" trong schema
        field_id=1000,            # ID mới cho partition (tự đặt, chỉ cần unique)
        transform="identity",     # Transform dạng string
        name="department_identity"
    )
)

# Tạo DB nếu chưa có
try:
    catalog.create_namespace(database_name)
    print('da create_namespace')
except Exception:
    print(f'da ton tai namespace {database_name}')
    pass

da ton tai namespace test


In [2]:
print(catalog.list_namespaces())

[('bronze',), ('default',), ('gold',), ('silver',), ('test',)]


In [3]:
identifier = "test.data_demo"
try:
    table = catalog.load_table(identifier)
    print("🎉 TABLE LOADED OK")
    print(table)
except Exception as e:
    print("❌ ERROR LOADING TABLE")
    print(e)

🎉 TABLE LOADED OK
data_demo(
  1: order_id: optional string,
  2: order_date: optional date,
  3: customer_id: optional string,
  4: customer_name: optional string,
  5: product_id: optional string,
  6: product_name: optional string,
  7: category: optional string,
  8: quantity: optional int,
  9: price: optional int,
  10: total_amount: optional int
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=5029198712650855573, schema_id=0


In [5]:
table_properties = {
    "format-version": "2",
    "write.metadata.delete-after-commit.enabled": "true"
}
# Tạo bảng
try:
    table = catalog.create_table(
        identifier=f"{database_name}.{table_name}",
        schema=schema,
        partition_spec=spec,
        properties=table_properties,
        # location=f"s3://warehouse/{database_name}.db/{table_name}"
    )
    print("Table created")
except Exception:
    table = catalog.load_table(f"{database_name}.{table_name}")
    print("Table already exists")

Table created


In [9]:
# Dữ liệu mẫu
records = [
    {"id": 1, "name": "Nguyen Van A", "age": 30, "department": "IT"},
    {"id": 2, "name": "Tran Thi B", "age": 25, "department": "HR"},
    {"id": 3, "name": "Le Van C", "age": 28, "department": "IT"},
]

import pyarrow as pa

correct_schema = pa.schema([
    pa.field("id", pa.int64(), nullable=False),   # required long
    pa.field("name", pa.string(), nullable=True),
    pa.field("age", pa.int32(), nullable=True),    # int32 instead of int64
    pa.field("department", pa.string(), nullable=True),
])

df = pa.Table.from_pylist(records)

# Cast dataframe
df = df.cast(correct_schema)

table.append(df)

print("Data written successfully!")

Data written successfully!


In [8]:
import pyiceberg
print(pyiceberg.__version__)

0.9.1


In [13]:
print(catalog.list_namespaces())
print(catalog.list_tables(database_name))

[('bronze',), ('default',), ('gold',), ('silver',), ('test',)]
[('test', 'test_iceberg_v1'), ('test', 'test_iceberg'), ('test', 'data_demo'), ('test', 'test_spark_kafka')]
